In [6]:
import cv2
import numpy as np
import rasterio
import os, glob

In [7]:
fp_T3 = r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/T3_2022_4326.tif"
with rasterio.open(fp_T3) as src:
    mask3_nodata = np.array([src.read_masks(1)])/255
fp_T4 = r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/RS_OK/T4_2022_4326_forest_oke.tif"
with rasterio.open(fp_T4) as src:
    meta = src.meta

In [ ]:
with rasterio.open(r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/xoa_thang_truoc_nodata/xoa_thang_truoc_di.tif", 'w', **meta) as dst:
    dst.write(mask3_nodata.astype('uint8'))

In [2]:
fp_T3 = r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/RS_OK/T3_2022_4326_forest_oke.tif"
fp_T4 = r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/RS_OK/T4_2022_4326_forest_oke.tif"

fp_them3vao4 = r"/home/skm/SKM16/Tmp/add3vao4.tif"
fp_them4vao3 = r"/home/skm/SKM16/Tmp/add4vao3.tif"
fp_xoa_het = r"/home/skm/SKM16/Tmp/xoa43___.tif"


In [3]:
with rasterio.open(fp_T3) as src:
    mask_t3 = src.read()*255
    mask3_nodata = np.array([src.read_masks(1)])/255
    meta = src.meta
with rasterio.open(fp_T4) as src:
    mask_t4 = src.read()*255
    mask4_nodata = np.array([src.read_masks(1)])/255

with rasterio.open(fp_them3vao4) as src:
    mask_34 = src.read()
    mask_34[mask_34!=0]=1
with rasterio.open(fp_them4vao3) as src:
    mask_43 = src.read()
    mask_43[mask_43!=0]=1
with rasterio.open(fp_xoa_het) as src:
    mask_xoa = src.read()
    mask_xoa[mask_xoa!=0]=1
    mask_xoa = 1-mask_xoa

In [4]:
mask_t3_them_t4 = mask_t3 + mask_43*mask_t4
mask_t3_them_t4[mask_t3_them_t4!=0]=255
mask_t3_them_t4 = mask_t3_them_t4*mask_xoa*mask3_nodata
mask_t3_them_t4[mask_t3_them_t4!=0]=1

mask_t4_them_t3 = mask_t4 + mask_34*mask_t3
mask_t4_them_t3[mask_t4_them_t3!=0]=255
mask_t4_them_t3 = mask_t4_them_t3*mask_xoa*mask4_nodata
mask_t4_them_t3[mask_t4_them_t3!=0]=1



In [5]:
with rasterio.open(fp_T3.replace('.tif', '_ok.tif'), 'w', **meta) as dst:
    dst.write(mask_t3_them_t4.astype('uint8'))
with rasterio.open(fp_T4.replace('.tif', '_ok.tif'), 'w', **meta) as dst:
    dst.write(mask_t4_them_t3.astype('uint8'))

# Them nuoc vao lan 2

In [17]:
import rasterio
import numpy as np

In [21]:
fp_img = r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/rs_water/T4_2022_4326.tif"
fp_add = r"/home/skm/SKM16/Tmp/mask_water_them_vao_lan2.tif"

In [22]:
with rasterio.open(fp_img) as src:
    mask_water = src.read()
    meta = src.meta
with rasterio.open(fp_add) as src:
    mask_add = src.read()


In [23]:
mask_water = mask_water + mask_add
mask_water[mask_water != 0] = 255
with rasterio.open(fp_img.replace('.tif','_oke.tif'), 'w', **meta) as dst:
    dst.write(mask_water.astype('uint8'))

In [36]:
import rasterio
import numpy as np

In [37]:
fp_img_mosaic = r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/T3_2022_4326.tif"
fp_plant = r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/RS/T3_2022_4326_plant.tif"
fp_forest = r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/RS/T3_2022_4326_forest.tif"
fp_water = r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/RS/T3_2022_4326_water.tif"

In [38]:
with rasterio.open(fp_img_mosaic) as src:
    mask_nodata = np.array([src.read_masks(1)])


with rasterio.open(fp_water) as src:
    mask_water = src.read()
    index_water = np.where(mask_water != 0)
    meta = src.meta

with rasterio.open(fp_plant) as src:
    mask_plant = src.read()
    mask_plant[mask_plant==255] = 1
    # mask_plant[mask_plant!=1] = 0
    # idx_plant = np.where(mask_plant == 1)




In [39]:
index_nodata = np.where(mask_nodata==0)
mask_plant[index_water] = 2
mask_plant[mask_plant == 0] = 3
mask_plant[index_nodata] = 0
with rasterio.open(r"/home/skm/SKM16/Work/Npark_planet2/img_uint8/RS/T3_2022_4326_color.tif",'w',**meta) as dst:
    dst.write(mask_plant)
    dst.write_colormap(1, {
                    0: (0,0,0, 0),
                    1: (34,139,34,0), #Green
                    2: (100, 149, 237, 0), #water
                    3: (101,67,33, 0)}) #Buildup

In [40]:
with rasterio.open(fp_forest) as src:
    mask_forest = src.read()
    mask_forest[mask_forest!=0] = 1
    meta = src.meta
with rasterio.open(fp_forest.replace(".tif","_oke.tif"),'w',**meta) as dst:
    dst.write(mask_forest.astype('uint8'))